# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from tqdm.notebook import tqdm

Загрузим все файлы в список для обеспечения возможности последующей обработки в цикле данных по всем регионам.

In [2]:
files = ['/datasets/geo_data_0.csv',
         '/datasets/geo_data_1.csv',
         '/datasets/geo_data_2.csv'] #сохраняю адреса файлов в список для последующей загрузки через цикл
datas = [] #в этот список буду добавлять датафреймы
for file in files:
    try:
        datas.append(pd.read_csv(file))
    except:
        print('Не удалось загрузить файл:', file)

Посморим на структуру данных, наличие пропусков и дубликатов.

In [3]:
for i in range(len(datas)):
    print('Данные по региону №', i + 1)
    print(datas[i].head())
    print('Пропуски:')
    print(datas[i].isna().mean())
    print('Дубликатов:')
    print(datas[i].duplicated().sum())
    print()

Данные по региону № 1
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
Пропуски:
id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64
Дубликатов:
0

Данные по региону № 2
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
Пропуски:
id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64
Дубликатов:
0

Данные по региону № 3
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 

## Обучение и проверка модели

Создадим функцию, которая получив на вход данные по региону, возвратит переменные с целевым признаком и остальными признаками.

In [4]:
def target_select(data):
    features = data.drop(['product', 'id'], axis=1)
    target = data['product']
    return features, target

Применив созданную функцию, также создадим функцию, которая будет обучать модель по данным региона и печатать средний запас сырья в скважине региона, средний запас сырья в скважине региона по предсказанным моделью данным, а также метрику RMSE; возвращать обученную модель.

In [5]:
def train_model(data):
    features, target = target_select(data)
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=123)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predictions)
    rmse = np.sqrt(mse)
    print('Средний запас сырья:', target.mean())
    print('Средний запас предсказанного сырья:', model.predict(features).mean())
    print('RMSE:', rmse)
    return model

В цикле пройдемся созданной функцией по данным всех регионов. 

Кроме того, создадим список, в котором будут храниться модели по каждому региону.

In [6]:
models = []
for i in range(len(datas)):
    print('Регион №', i + 1)
    model = train_model(datas[i])
    models.append(model)
    print()

Регион № 1
Средний запас сырья: 92.50000000000001
Средний запас предсказанного сырья: 92.42468449498216
RMSE: 37.64786282376176

Регион № 2
Средний запас сырья: 68.82500000000002
Средний запас предсказанного сырья: 68.8265765614402
RMSE: 0.8954139804944304

Регион № 3
Средний запас сырья: 95.00000000000004
Средний запас предсказанного сырья: 95.05627778732796
RMSE: 40.12803006598514



**Вывод**

Модели в целом предсказывают верный общий запас сырья в скважинах региона. Судя по метрике RMSE предсказания объема сырья точнее всего по второму региону.

## Подготовка к расчёту прибыли

С целью расчета прогнозируемой прибыли сохраним все ключевые значения в отдельных переменных.

In [7]:
BUDGET_FOR_REGION = 10_000_000_000
DEVELOPMENT_WELLS_NUM = 200
RESEARCHED_WELLS_NUM = 500
INCOME_PER_BARREL = 450
INCOME_PER_UNIT = INCOME_PER_BARREL * 1_000

Расчитаем минимальное среднее количество сырья, которое позволит вести разработку в регионе без убытков.

In [8]:
sufficient_raw = (BUDGET_FOR_REGION / DEVELOPMENT_WELLS_NUM) / INCOME_PER_UNIT
print('Необходимое минимальное среднее количество сырья в скважинах', round(sufficient_raw), 'тыс. баррелей')

Необходимое минимальное среднее количество сырья в скважинах 111 тыс. баррелей


Сравним данное значение со средними значениями реального количества сырья во всех скважинах региона.

In [9]:
for i in range(len(datas)):
    print('Регион №', i + 1)
    print('Среднее количество сырья в скважинах:', round(datas[i]['product'].mean()), 'тыс. баррелей')

Регион № 1
Среднее количество сырья в скважинах: 93 тыс. баррелей
Регион № 2
Среднее количество сырья в скважинах: 69 тыс. баррелей
Регион № 3
Среднее количество сырья в скважинах: 95 тыс. баррелей


**Вывод**

Среднее количество сырья в скважинах регионов меньше, чем необходимое для безубыточной разработки скважин. В связи с этим случайный выбор разрабатываемых скважин скорее всего приведет к убытка. Таким образом, необходимо отбирать скважины и регионы для разработки с помощью анализа данных и машинного обучения.

## Расчёт прибыли и рисков 

Создадим функцию, которая будет производить расчет реально полученной прибыли от отобранных моделью скважин.

In [10]:
def profit_calc(data_index): #функция будет принимать индекс данных из списка предоставленных данных по регионам 
    features, target = target_select(datas[data_index]) #выделяем целевой признак ранее созданной функцией
    predictions = models[data_index].predict(features) #сохраняем предсказания модели в переменную
    predictions = pd.Series(predictions).sort_values(ascending=False)[:200] #отбираем 200 наиболее прибольных скважин
    total_income = target[predictions.index].sum() * INCOME_PER_UNIT #находим прибыль от данных скважин
    profit = total_income - BUDGET_FOR_REGION
    return profit

Применим данную функцию к данным по каждому региону.

In [11]:
print('Прибыль от отобранных моделью скважин')
for i in range(len(datas)):
    print(f'Региона № {i + 1}: {profit_calc(i) * 10 ** (-9):.4} млрд. руб.')

Прибыль от отобранных моделью скважин
Региона № 1: 3.492 млрд. руб.
Региона № 2: 2.415 млрд. руб.
Региона № 3: 2.596 млрд. руб.


Создадим функцию, которая будет отбирать 500 случайных скважин региона, отбирать из них 200 наиболее прибыльных по предсказаниям модели и рассчитывать прибыль на основании реальных данных о количестве сырья в скважине; через bootstrap обработает 1000 вариантов случайных выборок, найдет по ним среднюю прибыль, 95%-й доверительный интервал и вероятность убытков.

In [12]:
state = RandomState(12345)

In [13]:
def estimate_risks(data_index, n = 1000):
    profits = [] #в данный список буду сохранять значения прибыли
    for i in tqdm(range(n)):
        researched_wells = datas[data_index].sample(RESEARCHED_WELLS_NUM, random_state=state) 
        #делаю случайную выборку исследуемых скважин
        researched_wells = researched_wells.reset_index(drop=True) 
        #сбрасываю индексы для последующего сопоставления с индексами предсказаний
        features, target = target_select(researched_wells)
        #разделяю данные об исследуемых скважинах на целевой признак и признаки для обучения модели
        predictions = models[data_index].predict(features)
        #использую модель, обученную по данным региона, для предсказаний по отобранным скважинам
        predictions = pd.Series(predictions).sort_values(ascending=False)[:DEVELOPMENT_WELLS_NUM]
        #отбираю наиболее прибыльные скважины по результатам предсказания модели
        total_income = target[predictions.index].sum() * INCOME_PER_UNIT 
        #считаю общий доход исходя из реального объема сырья в скважинах
        profit = total_income - BUDGET_FOR_REGION #считаю прибыль
        profits.append(profit) #сохраняю значения прибыли в список
    profits = pd.Series(profits)
    mean_profit = profits.mean() #нахожу среднюю прибыль по получившимся значениями прибыли
    confidence_interval = (profits.quantile(0.025), profits.quantile(0.975)) #нахожу 95%-й доверительный интервал
    loss_prob = (profits < 0).mean() #нахожу долю убыточных значений
    print('Риски по региону №', data_index + 1)
    print(f'Средняя прибыль из возможных: {mean_profit * 10 ** (-9):.3} млрд.руб.')
    print(f'95%-й доверительный интервал возможной прибыли: от {confidence_interval[0] * 10 ** (-9):.3f} млрд. руб. до {confidence_interval[1] * 10 ** (-9):.3f} млрд.руб.')
    print('Вероятность убытков:', loss_prob)

In [14]:
print('Количество изучаемых регионов:', len(datas))
for index in range(len(datas)):
    estimate_risks(index)
    print()

Количество изучаемых регионов: 3



Риски по региону № 1
Средняя прибыль из возможных: 0.42 млрд.руб.
95%-й доверительный интервал возможной прибыли: от -0.088 млрд. руб. до 0.908 млрд.руб.
Вероятность убытков: 0.056




Риски по региону № 2
Средняя прибыль из возможных: 0.457 млрд.руб.
95%-й доверительный интервал возможной прибыли: от 0.067 млрд. руб. до 0.858 млрд.руб.
Вероятность убытков: 0.01




Риски по региону № 3
Средняя прибыль из возможных: 0.356 млрд.руб.
95%-й доверительный интервал возможной прибыли: от -0.155 млрд. руб. до 0.863 млрд.руб.
Вероятность убытков: 0.104



# Общий вывод

В результате анализа данных об объеме сырья в скважинах регионов установлено, что средний запас сырья в скважинах второго региона меньше чем у остальных (68 тыс. баррелей против 92 - в первом регионе и 95 - в третьем).

С целью определения возможности предсказания объема сырья в месторожднении регонов обучены модели линейной регрессии для получения предсказаний объема сырья в скважинах на основании признаков месторождения. 

По второму региону модель предсказывает наиболее точные значения объема сырья в месторождении (метрика RMSE - менее 1, в отличии от первого и второго региона, где RMSE равно 37 и 40 соответственно). 

Оценка рисков разработки месторождений в регионах методом bootstrap на 1000 вариантах случайных выборок показала наиболее выгодные перспективы разработки месторождений второго региона. В пользу это свидетельствуют следующие показатели: 
- вероятность убыточности разработки скважин второго региона всего лишь около одного процента (по первому и третьему регионам вероятность убытков около 5-8 процентов), 
- 95%-й доверительный интервал возможной прибыли с регоина находится в пределах от 0.057 млрд.руб. до 0.850 млрд.руб. (по другим регионам доходит до отрицательных значений - убытков);
- средняя прибыль из всех полученных по второму региону вариантов прибыли - 0.447 млрд.руб. (в первом - 0.41, в третьем - 0.372).